In [62]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing

In [63]:
# Add funtion to sys.path
os.chdir(pb.__path__[0]+'/..')   
# pb.__path__[0] allow us to location path without concerning different machine
import sys  
sys.path.append(os.path.join(os.getcwd(),'wip\Rio_Code\Fun_P2'))  
# sys.path.append(os.path.join(pb.__path__[0]+'/..','wip\Rio_Code\Fun_P2'))  
from Fun_P2_Union_crack import (
    GetScan,
    Run_model_w_dry_out ,
    Run_model_wo_Dry_out,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,)

In [134]:
# all values here must be a list, even it is a single object
Para_dict_All = {
   "Para_Set":[ "OKane2022",],
   "electrolyte":["lipf6_Valoen2005",],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],
   "Negative electrode diffusivity [m2.s-1]":[2e-13,],
   "Positive electrode diffusivity [m2.s-1]":[1e-13,],
   # general SEI
   'SEI resistivity [Ohm.m]':[2E5,1e5,],
   # solvent-diffusion limited
   'Outer SEI solvent diffusivity [m2.s-1]':[1.7e-22,],
   'Bulk solvent concentration [mol.m-3]':[4541.0,],
   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-12,],    
   'Lithium interstitial reference concentration [mol.m-3]':[15,20,],
   # ec-reaction limited
   'EC diffusivity [m2.s-1]':[2e-20,],
   'SEI kinetic rate constant [m.s-1]':[1e-16,],
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-6,],
   'Lithium plating kinetic rate constant [m.s-1]':[1E-10,],
   # Crack model
   "Negative electrode LAM constant proportional term [s-1]":[ 2.7778e-08,],
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-08,],
   # make it simple for now,], but may want to have T dependency in the future
   "Negative electrode cracking rate":[ 3.9e-20,],
   "Positive electrode cracking rate":[ 3.9e-20,],
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
}

def recursive_scan(mylist,kvs, key_list, acc):
    # 递归终止条件
    if len(key_list) == 0:
        mylist.append(acc.copy())   # copy.deepcopy(acc) 如果value是个list，就要deep copy了
        return mylist
    # 继续递归
    k = key_list[0]
    for v in kvs[k]:
        acc[k] = v
        # print(v)
        recursive_scan(mylist,kvs, key_list[1:], acc)
# this function is to initialize the para with a known dict
def Para_init(Para_dict):
    ChemistryChen=getattr(pb.parameter_sets,Para_dict["Para_Set"]) 
    Para_dict.pop("Para_Set")
    if Para_dict.__contains__("electrolyte"):
        ChemistryChen["electrolyte"] = Para_dict["electrolyte"]  
        Para_dict.pop("electrolyte")
    Para_0=pb.ParameterValues(chemistry=ChemistryChen)
    for key, value in Para_dict.items():
        # risk: will update parameter that doesn't exist, so need to make sure the name is right 
        Para_0.update({key: value},check_already_exists=False)
    return Para_0

Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})



# 至此获得需要扫描的参数list Para_dict_list
#  Run_model_w_dry_out may need to chaneg to scan_model_wwo_dryout, 
# as input change from DatePack_scan_i to Para_dict_All
#  

In [131]:
# Test this branch - Coupled_RioP2 newly created from Simon's develop branch
# Run without dry-out model
V_max = 4.2;        V_min = 2.5;   Temper_RPT = 25; 
Total_Cycles = 10; save_at_cycles = 1;  
for i in range(0,1):  # Key scan parameters:
    Diff_SEI_i = 1.7e-22; 
    R_SEI_i = 2E5;   Bulk_Sol_Con_i = 4541.0;
    D_Li_inSEI_i = 1e-12;    # default: 1e-20 
    c_Li_inte_ref_i = 15;    # default: 15
    Diff_EC_i   = 2e-20;       k_SEI_i   = 1e-16;
    LAMcr_prop_i=2.7778e-08;   Crack_rate_i=3.9e-20; 
    Couple_SEI_LiP_i = 1e-6; # default: 1e-6
    k_LiP_i = 1E-10;         # default: 1e-10
    Temper = 25;
    mesh_par = 30;
     
for i in range(0,1):  
    ChemistryChen=pb.parameter_sets.OKane2022   
    #ChemistryChen["electrolyte"] = "lipf6_Valoen2005"
    Para_0=pb.ParameterValues(chemistry=ChemistryChen)
    # DFN parameter
    Para_0.update({"Upper voltage cut-off [V]": 4.21})
    Para_0.update({"Lower voltage cut-off [V]": 2.49})
    Para_0.update({"Negative electrode diffusivity [m2.s-1]": 2e-13})
    Para_0.update({"Positive electrode diffusivity [m2.s-1]": 1e-13})
    # general SEI
    Para_0.update({'SEI resistivity [Ohm.m]':R_SEI_i}) ;
    # solvent-diffusion limited
    Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI_i});
    Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con_i});
    # interstitial-diffusion limited
    Para_0.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':D_Li_inSEI_i})       
    Para_0.update({'Lithium interstitial reference concentration [mol.m-3]':c_Li_inte_ref_i}) 
    # ec-reaction limited
    Para_0.update({'EC diffusivity [m2.s-1]':Diff_EC_i}) 
    Para_0.update({'SEI kinetic rate constant [m.s-1]':k_SEI_i}) 
    Para_0.update({'EC initial concentration in electrolyte [mol.m-3]':Bulk_Sol_Con_i}) 
    # LiP and coupling with SEI:
    Para_0.update({"Dead lithium decay constant [s-1]": Couple_SEI_LiP_i})   
    Para_0.update({'Lithium plating kinetic rate constant [m.s-1]':k_LiP_i})  
    # Crack model
    Para_0.update({"Negative electrode LAM constant proportional term [s-1]": LAMcr_prop_i})
    Para_0.update({"Positive electrode LAM constant proportional term [s-1]": LAMcr_prop_i})  
    # make it simple for now, but may want to have T dependency in the future
    Para_0.update({"Negative electrode cracking rate": Crack_rate_i})
    Para_0.update({"Positive electrode cracking rate": Crack_rate_i})
    Para_0.update({"Negative electrode volume change": 0.0})
    Para_0.update({"Positive electrode volume change": 0.0}) 
    Model_0 = pb.lithium_ion.DFN(     ) #
    var = pb.standard_spatial_vars  
    var_pts = {
        var.x_n: 20,  
        var.x_s: 10,  
        var.x_p: 20,  
        var.r_n: mesh_par,  
        var.r_p: mesh_par,  }

In [ ]:
# Run break-in cycle step by step:

Experiment_Breakin_0 = pb.Experiment( [
    ( f"Discharge at 1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.3C until {V_max} V" )  ] *3 )
# Run 1st cycle
Sim_Breakin_0    = pb.Simulation(
    Model_0,        experiment = Experiment_Breakin_0,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_0    = Sim_Breakin_0.solve(calc_esoh=False)

In [ ]:
def GetSol_dict (my_dict, keys_all, Sol_Breakin_0, 
    cycle_no,step_CD, step_CC , step_RE, step_CV ):

    [keys_loc,keys_tim,keys_cyc]  = keys_all;
    # get time_based variables:
    if len(keys_tim): 
        for key in keys_tim:
            step_no = eval("step_{}".format(key[0:2]))
            if key[3:] == "Time [h]":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries
                    -
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries[0])
            else:
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries)
    # get cycle_step_based variables:
    if len(keys_cyc): 
        for key in keys_cyc:
            if key in ["Discharge capacity [A.h]"]:
                step_no = step_CD;
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[-1]
                    - 
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[0])
            elif key[0:5] in ["CDend","CCend","CVend","REend",
                            "CDsta","CCsta","CVsta","REsta",]:
                step_no = eval("step_{}".format(key[0:2]))
                if key[2:5] == "sta":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[0])
                elif key[2:5] == "end":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[-1])
                
    # get location_based variables:
    if len(keys_loc): 
        for key in keys_loc:
            if key in ["x_n [m]","x [m]","x_s [m]","x_p [m]"]:
                #print("These variables only add once")
                if not len(my_dict[key]):   # special: add only once
                    my_dict[key] = Sol_Breakin_0[key].entries[:,-1]
            elif key[0:5] in ["CDend","CCend","CVend","REend",
                            "CDsta","CCsta","CVsta","REsta",]:       # special: need to start from 0
                #print("These variables add multiple times")
                step_no = eval("step_{}".format(key[0:2]))
                if key[2:5] == "sta":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,0])
                elif key[2:5] == "end":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,-1])
    return my_dict           

In [ ]:
# design the function:
# input: sol; list of varibales that want to save; 
# output: dict of varbales and results which can be saved as .mat;
# define three types of keys: location related; time related; cycle related
# Function input:
keys_loc = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    "REend Total SEI on cracks thickness [m]"

]
keys_tim = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "RE Terminal voltage [V]",
]
keys_cyc = [   # default: CDend
    "Discharge capacity [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    "CDend X-averaged total SEI on cracks thickness [m]",
    "CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode SOC", 
    "CDend Negative electrode SOC",
    "CDsta Positive electrode SOC", 
    "CDend Positive electrode SOC",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]
keys_all = [keys_loc,keys_tim,keys_cyc];


In [ ]:
my_dict = {}; # initialize my_dict
for keys in keys_all:
    for key in keys:
        my_dict[key]=[];
cycle_no = -1; step_CD = 0;  step_CC = 2; step_RE =1; step_CV =3
my_dict_RPT = GetSol_dict (my_dict,keys_all, Sol_Breakin_0,
 cycle_no, step_CD ,  step_CC, step_RE, step_CV  )

In [ ]:
my_dict_RPT

{'x [m]': array([2.1300e-06, 6.3900e-06, 1.0650e-05, 1.4910e-05, 1.9170e-05,
        2.3430e-05, 2.7690e-05, 3.1950e-05, 3.6210e-05, 4.0470e-05,
        4.4730e-05, 4.8990e-05, 5.3250e-05, 5.7510e-05, 6.1770e-05,
        6.6030e-05, 7.0290e-05, 7.4550e-05, 7.8810e-05, 8.3070e-05,
        8.5800e-05, 8.7000e-05, 8.8200e-05, 8.9400e-05, 9.0600e-05,
        9.1800e-05, 9.3000e-05, 9.4200e-05, 9.5400e-05, 9.6600e-05,
        9.9090e-05, 1.0287e-04, 1.0665e-04, 1.1043e-04, 1.1421e-04,
        1.1799e-04, 1.2177e-04, 1.2555e-04, 1.2933e-04, 1.3311e-04,
        1.3689e-04, 1.4067e-04, 1.4445e-04, 1.4823e-04, 1.5201e-04,
        1.5579e-04, 1.5957e-04, 1.6335e-04, 1.6713e-04, 1.7091e-04]),
 'x_n [m]': array([2.130e-06, 6.390e-06, 1.065e-05, 1.491e-05, 1.917e-05, 2.343e-05,
        2.769e-05, 3.195e-05, 3.621e-05, 4.047e-05, 4.473e-05, 4.899e-05,
        5.325e-05, 5.751e-05, 6.177e-05, 6.603e-05, 7.029e-05, 7.455e-05,
        7.881e-05, 8.307e-05]),
 'x_s [m]': array([8.58e-05, 8.70e-05, 8.82e

In [ ]:
# now has this problem: the RPT results and ageing results may contradict
# to solve this problems, now just avoid defining the same variables for both
# or: avoid merging the two dict together
x = {'a': 1, 'b': 2}
y = {'b': 3, 'c': 4}
z = {**x, **y}

In [ ]:
print(z)

{'a': 1, 'b': 3, 'c': 4}


In [ ]:
Model_0 = pb.lithium_ion.DFN(     
    options={
        "particle": "Fickian diffusion",          
        "SEI":"solvent-diffusion limited",   
        "SEI on cracks":"true",  
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",      
        "particle mechanics":"swelling and cracking",  
        "loss of active material":"stress-driven", 
        "lithium plating":"partially reversible"      } ) #
Model_0.initial_conditions.items()

In [ ]:
list_rio = []
for var, equation in Model_0.initial_conditions.items():
    #print(var._name)
    list_rio.append(var._name)
# delete Porosity times concentration and Electrolyte potential then add back
list_rio.remove("Porosity times concentration");
list_rio.remove("Electrolyte potential");
list_rio.extend(
    ("Negative electrode porosity times concentration",
    "Separator porosity times concentration",
    "Positive electrode porosity times concentration",   

    "Negative electrolyte potential",
    "Separator electrolyte potential",
    "Positive electrolyte potential",))


In [ ]:
for list_rio_i in list_rio:
    print(list_rio_i)

In [ ]:
for var, equation in Model_0.initial_conditions.items():
    print(var)
    #list_rio.append(var)

In [ ]:
list_rio[1]._name

In [ ]:
# get a new run_model_based_on_last_solution:
# update 220808 to satisfy random model option:
def Run_Model_Base_On_Last_Solution( 
    Model  , Sol , Para_update, ModelExperiment, 
    Update_Cycles,Temper_i ,mesh_par):
    # Use Sulzer's method: inplace = false
    # Important line: define new model based on previous solution
    Ratio_CeLi = Para_update["Ratio of Li-ion concentration change in electrolyte consider solvent consumption"]
    dict_short = {}; 
    list_short = []
    # update 220808 to satisfy random model option:
    for var, equation in Model_0.initial_conditions.items():
        #print(var._name)
        list_short.append(var._name)
    # delete Porosity times concentration and Electrolyte potential then add back
    list_short.remove("Porosity times concentration");
    list_short.remove("Electrolyte potential");
    list_short.extend(
        ("Negative electrode porosity times concentration",
        "Separator porosity times concentration",
        "Positive electrode porosity times concentration",   

        "Negative electrolyte potential",
        "Separator electrolyte potential",
        "Positive electrolyte potential",))
    for list_short_i in list_short:
        dict_short.update( { list_short_i : Sol.last_state[list_short_i].data  }  )
    dict_short["Negative electrode porosity times concentration"] = (
        dict_short["Negative electrode porosity times concentration"] * Ratio_CeLi )# important: update sol here!
    dict_short["Separator porosity times concentration"] = (
        dict_short["Separator porosity times concentration"] * Ratio_CeLi )# important: update sol here!
    dict_short["Positive electrode porosity times concentration"] = (
        dict_short["Positive electrode porosity times concentration"] * Ratio_CeLi )# important: update sol here!
    Model_new = Model.set_initial_conditions_from(dict_short, inplace=False)
    Para_update.update(   {'Ambient temperature [K]':273.15+Temper_i });   # run model at 45 degree C
    
    var = pb.standard_spatial_vars  
    var_pts = {var.x_n: 20,  var.x_s: 10,  var.x_p: 20,  var.r_n: int(mesh_par),  var.r_p: int(mesh_par),  }
    
    Simnew = pb.Simulation(
        Model_new,
        experiment = ModelExperiment, 
        parameter_values=Para_update, 
        solver = pb.CasadiSolver(),
        var_pts = var_pts,
    )
    Sol_new = Simnew.solve(calc_esoh=False,save_at_cycles = Update_Cycles) # save every several cycles, can save RAM greatly
    # print("Solved this model in {}".format(ModelTimer.time()))
    return Model_new, Sol_new